# Stage I: Physical Features for KMeans Clustering

This notebook builds the physical feature matrix for Stage I KMeans clustering by querying DuckDB views.

**Features:**
- `age`: years since installation (2024 – installation year, with fractional precision)
- `diameter`: physical diameter of the meter (`DIAM_COMP`)
- `canya`: proxy for accumulated consumption (median of yearly averages × age)
- `brand_model`: joint categorical label for `MARCA_COMP` + `CODI_MODEL` (one-hot encoded)

**Normalization:**
- `age` and `diameter`: min-max scaling [0, 1]
- `canya`: z-score (standardization, mean=0, std=1)
- `brand_model`: one-hot encoding (27 categories)


In [1]:
# Imports and constants
from __future__ import annotations

from pathlib import Path
from typing import Iterable, Tuple

import duckdb
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder

CURRENT_YEAR = 2024
DEFAULT_DB_PATH = Path("..") / "analytics.duckdb"  # Relative to notebook location
NUMERIC_FEATURES = ["age", "diameter", "canya"]


## Function: Compute Physical Features

Queries DuckDB to extract raw physical features for domestic meters.


In [2]:
def compute_physical_features(
    *,
    db_path: str | Path = DEFAULT_DB_PATH,
    current_year: int = CURRENT_YEAR,
) -> pd.DataFrame:
    """
    Compute physical features required by Stage I.

    Parameters
    ----------
    db_path:
        Path to ``analytics.duckdb`` containing the required views.
    current_year:
        Reference year used to compute meter age. Defaults to 2024.

    Returns
    -------
    pandas.DataFrame
        DataFrame indexed by meter id with the columns:
        ``age``, ``diameter``, ``canya``, ``brand_model``.
    """

    path = Path(db_path)
    if not path.exists():
        raise FileNotFoundError(
            f"DuckDB database not found at {path}. "
            "Run data/create_database.py to generate analytics.duckdb."
        )

    con = duckdb.connect(database=str(path), read_only=True)

    sql = """
    WITH domestic AS (
        SELECT
            cm."POLIZA_SUMINISTRO"::VARCHAR AS meter_id,
            CAST(cm.DATA_INST_COMP AS DATE) AS installation_date,
            CAST(cm.DIAM_COMP AS DOUBLE) AS diameter,
            CAST(cm.MARCA_COMP AS VARCHAR) AS marca_comp,
            CAST(cm.CODI_MODEL AS VARCHAR) AS codi_model,
            CAST(cd.FECHA AS DATE) AS fecha,
            cd.CONSUMO_REAL
        FROM counter_metadata cm
        JOIN consumption_data cd
            ON cm."POLIZA_SUMINISTRO" = cd."POLIZA_SUMINISTRO"
        WHERE cm.US_AIGUA_GEST = 'D'
    ),
    metadata AS (
        SELECT
            meter_id,
            MIN(installation_date) AS installation_date,
            MIN(diameter) AS diameter,
            MIN(marca_comp) AS marca_comp,
            MIN(codi_model) AS codi_model
        FROM domestic
        GROUP BY meter_id
    ),
    yearly AS (
        SELECT
            meter_id,
            EXTRACT(YEAR FROM fecha) AS year,
            AVG(CONSUMO_REAL) AS avg_consumption
        FROM domestic
        GROUP BY meter_id, year
    ),
    avg_yearly AS (
        SELECT
            meter_id,
            AVG(avg_consumption) AS avg_yearly
        FROM yearly
        GROUP BY meter_id
    ),
    median_yearly AS (
        SELECT
            meter_id,
            MEDIAN(avg_consumption) AS median_yearly
        FROM yearly
        GROUP BY meter_id
    )
    SELECT
        m.meter_id,
        m.installation_date,
        m.diameter,
        m.marca_comp,
        m.codi_model,
        COALESCE(a.avg_yearly, 0) AS avg_yearly,
        COALESCE(md.median_yearly, 0) AS median_yearly
    FROM metadata m
    LEFT JOIN avg_yearly a USING (meter_id)
    LEFT JOIN median_yearly md USING (meter_id)
    """

    df = con.execute(sql).df()
    con.close()

    if df.empty:
        raise ValueError("No domestic meters found with the specified filter.")

    df["installation_date"] = pd.to_datetime(df["installation_date"], errors="coerce")

    reference_date = pd.Timestamp(year=current_year, month=12, day=31)
    days_since_install = (reference_date - df["installation_date"]).dt.days
    df["age"] = (days_since_install / 365.25).clip(lower=0)
    df["diameter"] = df["diameter"].astype(float)

    df["canya"] = df["median_yearly"].fillna(0) * df["age"]

    df["marca_comp"] = df["marca_comp"].astype(str).str.strip()
    df["codi_model"] = df["codi_model"].astype(str).str.strip()
    df["brand_model"] = (
        df["marca_comp"].fillna("UNK") + "::" + df["codi_model"].fillna("UNK")
    )

    final_columns: Iterable[str] = [
        "meter_id",
        "age",
        "diameter",
        "canya",
        "brand_model",
        "avg_yearly",
        "median_yearly",
    ]

    df = df.loc[:, final_columns]

    return df


## Function: Build Stage I Feature Matrix

Constructs the normalized + one-hot encoded feature matrix for Stage I KMeans.


In [3]:
def build_stage1_feature_matrix(
    *,
    db_path: str | Path = DEFAULT_DB_PATH,
    current_year: int = CURRENT_YEAR,
) -> Tuple[pd.DataFrame, MinMaxScaler, StandardScaler, OneHotEncoder]:
    """
    Construct the normalized + one-hot encoded feature matrix for Stage I.
    
    Uses min-max scaling for age and diameter, z-score (standardization) for canya.

    Returns
    -------
    tuple
        (features_df, fitted_minmax_scaler, fitted_standard_scaler, fitted_onehot_encoder)
    """

    raw_features = compute_physical_features(db_path=db_path, current_year=current_year)

    # Min-max scaling for age and diameter
    minmax_scaler = MinMaxScaler()
    age_diameter_scaled = minmax_scaler.fit_transform(raw_features[["age", "diameter"]])
    age_diameter_df = pd.DataFrame(
        age_diameter_scaled, 
        columns=["age", "diameter"], 
        index=raw_features.index
    )
    
    # Z-score (standardization) for canya
    standard_scaler = StandardScaler()
    canya_scaled = standard_scaler.fit_transform(raw_features[["canya"]])
    canya_df = pd.DataFrame(
        canya_scaled,
        columns=["canya"],
        index=raw_features.index
    )
    
    # Combine scaled numeric features
    scaled_df = pd.concat([age_diameter_df, canya_df], axis=1)

    encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
    brand_encoded = encoder.fit_transform(raw_features[["brand_model"]])
    brand_columns = [f"brand_model__{cat}" for cat in encoder.categories_[0]]
    brand_df = pd.DataFrame(brand_encoded, columns=brand_columns, index=raw_features.index)

    features = pd.concat(
        [
            raw_features[["meter_id"]],
            scaled_df,
            brand_df,
        ],
        axis=1,
    )

    return features, minmax_scaler, standard_scaler, encoder


In [7]:
# Build the feature matrix
features, minmax_scaler, standard_scaler, encoder = build_stage1_feature_matrix()

# Summary
num_brand_cols = sum(col.startswith("brand_model__") for col in features.columns)
print(f"Feature matrix shape: {features.shape}")
print(f"Numeric columns: {NUMERIC_FEATURES}")
print(f"Brand model OHE columns: {num_brand_cols}")
print(f"\nTotal domestic meters: {len(features):,}")


Feature matrix shape: (10427, 21)
Numeric columns: ['age', 'diameter', 'canya']
Brand model OHE columns: 17

Total domestic meters: 10,427


## Preview: Feature Matrix

Display first few rows and basic statistics.


In [8]:
# Preview the feature matrix
features.head(10)


,meter_id,age,diameter,canya,brand_model__IBE::66,brand_model__IBE::73,brand_model__ITR::2,brand_model__ITR::21,brand_model__ITR::22,brand_model__ITR::23,...,brand_model__ITR::45,brand_model__ITR::48,brand_model__ITR::50,brand_model__ITR::63,brand_model__SPL::2,brand_model__SPL::4,brand_model__ZAR::19,brand_model__ZAR::21,brand_model__ZAR::27,brand_model__ZAR::46
0,6OJDQQGEGAMQMCEI,0.483604,0.0,0.622863,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,6RJFPTINIGREZJAW,0.535960,0.0,3.553682,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,6UANFQ7M7FGWUD5B,0.423257,0.0,0.404061,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,DNK5DUHU3PNRSWM7,0.392670,0.0,2.715938,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,DRYASP6JN6SW4RGQ,0.558556,0.0,-0.704303,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
5,GMEISXX32QSUYLY3,0.131992,0.0,-0.836774,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
6,GNGS2SDZ6RXEIIX2,0.407826,0.0,-0.861846,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,J5RZ4JFXX2IV7VDK,0.159273,0.0,-0.924516,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8,J6RT7MCWQ3D75X7X,0.310554,0.0,0.303343,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
9,LSO5NSDK3YKOLXPF,0.387710,0.0,-0.609597,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [9]:
# Statistics for numeric features
features[["age", "diameter", "canya"]].describe()


,age,diameter,canya
count,10427.000000,10427.000000,1.042700e+04
mean,0.394483,0.001566,-1.839902e-17
std,0.144160,0.028603,1.000048e+00
min,0.000000,0.000000,-1.329960e+00
25%,0.321438,0.000000,-5.831638e-01
50%,0.420502,0.000000,-1.573206e-01
75%,0.484982,0.000000,3.891224e-01
max,1.000000,1.000000,5.351260e+01


## Visualization (Optional)

Explore the feature distributions and relationships.


In [ ]:
# Uncomment to visualize (requires matplotlib/seaborn)
# import matplotlib.pyplot as plt
# import seaborn as sns

# # Distribution of numeric features
# features[["age", "diameter", "canya"]].hist(bins=30, figsize=(12, 4))
# plt.tight_layout()
# plt.show()

# # Pairwise relationships
# sns.pairplot(features[["age", "diameter", "canya"]])
# plt.show()
